In [7]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

<h1>Política de inventario optima para el problema multi-periodo y multi-producto con abastecimiento limitado por capacidad , lotes mínimos y métas de inventario</h1>
<p>El siguiente modelo propone una formulación matemática para resolver el problema de definir las órdenes de abastecimiento óptimas para un sistema de inventario donde existe un proveedor con capacidad limitada, lotes mínimos de compra/abastecimiento, y metas de inventario.</p>
<h2>Modelo</h2>

In [3]:
model = LpProblem(name="Inventory-policies-problem", sense=LpMinimize)

<h3>Conjuntos:</h3>
<p>$t$: periodos {1,2,..,n}</p>
<p>$p$: productos {1,2,..,m}</p>
<h3>Parámetros:</h3>
<p>$C$: Costo de colocar la orden, independiente de las cantidades de los productos $p$ comprados</p>
<p>$L$: Lead time del proveedor</p>
<p>$T_{t}$: Capacidad del proveedor durante el periodo $t$</p>
<p>$Q_{p}$: Inventario inicial del producto $p$</p>
<p>$H_{p}$: Costo de mantenimiento del inventario en la bodega del producto $p$</p>
<p>$S_{p}$: Lote mínimo de pedido para el producto $p$</p>
<p>$W_{p}$: Consumo de capacidad del proveedor del producto $p$</p>
<p>$F_{pt}$: Ventas pronosticadas del producto $p$ durante el periodo $t$</p>
<p>$A_{pt}$: Llegadas proyectadas del producto $p$ durante el periodo $t$</p>
<p>$O_{pt}$: Meta de inventario para el producto $p$ al final del periodo $t$</p>
<p>$D_{pt}$: Costo por no cumplor la meta de inventario del producto $p$ al final del periodo $t$</p>

In [3]:
product_count = 500
periods_count = 52
# declarando los conjuntos
t_sets = [x for x in range(periods_count)]
p_sets = [x for x in range(product_count)]
# declarando los parámetros
c_parameter = 2500
l_parameter = 12
t_parameter = np.array([500 for x in range(periods_count)])
q_parameter = np.random.normal(loc=25.0, scale=5, size=product_count)
h_parameter = np.array([0.1 for x in range(product_count)])
s_parameter = np.ones(product_count)*25
w_parameter = np.ones(product_count)*2
f_parameter = [np.random.normal(loc=5, scale=0.1, size=product_count) for x in t_sets]
a_parameter = [np.zeros(product_count) for x in t_sets]
o_parameter = [np.ones(product_count)*15 for x in t_sets]
d_parameter = [np.ones(periods_count)*x for x in p_sets]
m_parameter = np.sum(f_parameter)**2

In [4]:
parameters_df = pd.read_excel('D:\OneDrive - Proquinal\Proyecto_Confort\SuggestedProgram_20210611\Analisis.xlsm', sheet_name='Overview', skiprows=5)

In [ ]:
t_parameter = np.array([26000 for x in range(periods_count)])
q_parameter = 

In [12]:
for column in range(len(parameters_df.columns)):
    print(column, parameters_df.columns[column])

0 Item
1 Semanas de corrección
2 SS
3 Min BatchSize
4 Inv.Policy
5 CSL
6 EOWT
7 Supplier
8 LeadTime Weeks
9 Older Open Orders
10 Arrivals 0
11 Arrivals 1
12 Arrivals 2
13 Arrivals 3
14 Arrivals 4
15 Arrivals 5
16 Arrivals 6
17 Arrivals 7
18 Arrivals 8
19 Arrivals 9
20 Arrivals 10
21 Arrivals 11
22 Arrivals 12
23 Arrivals 13
24 Arrivals 14
25 Arrivals 15
26 Arrivals 16
27 Arrivals 17
28 Arrivals 18
29 Arrivals 19
30 Arrivals 20
31 Arrivals 21
32 Arrivals 22
33 Arrivals 23
34 Arrivals 24
35 Total arrivals
36 --
37 Last week IOH
38 Current week IOH
39 Next weeks IOH
40 Current week +2
41 Current week +3
42 Current week +4
43 Current week +5
44 Current week +6
45 Current week +7
46 Current week +8
47 Current week +9
48 Current week +10
49 Current week +11
50 Current week +12
51 Current week +13
52 Current week +14
53 Current week +15
54 Current week +16
55 Current week +17
56 Current week +18
57 Current week +19
58 Current week +20
59 Current week +21
60 Current week +22
61 Current week +2

In [15]:
arrivals = np.array(parameters_df.iloc[:,63:89])
porders = np.array(parameters_df.iloc[])

In [21]:
arrivals[0]

array([ 3723.76666667,  3723.76666667,  3704.21935484,  3586.93548387,
        3586.93548387,  3586.93548387,  3586.93548387,  3610.48387097,
        3619.90322581,  3619.90322581,  3619.90322581,  3686.51182796,
        3853.03333333,  3853.03333333,  3853.03333333,  3853.03333333,
        3754.03225806,  3754.03225806,  3754.03225806,  3754.03225806,
        3733.27096774,  3717.7       ,  3717.7       ,  3717.7       ,
        3701.11290323, 92671.94623656])

<h3>Variables:</h3>
<p>$X_{pt}$ :Cantidad a ordenar del producto $p$ al comienzo del periodo $t$</p>
<p>$K_{pt}$ :Inventario on hand del producto $p$ al final del periodo $t$</p>
<p>$P_{t}$ :Cantidad de Pedidos a colocar al inicio del periodo $t$. Esta variable determina cuántos contenedores se van a despachar por el proveedor</p>
<p>$R_{pt}=\left\lbrace\begin{array}{c} 1 \text{ si el producto p será puesto en la orden de compra al inicio del periodo t} \\ 0 \text{ en otro caso} \end{array}\right.$</p>
<p>$E_{pt}=\left\lbrace\begin{array}{c} 1 \text{si se permitirá que el producto p quede por debajo de su meta de inventario al final del periodo t} \\ 0~\text{en otro caso} \end{array}\right.$</p>

In [4]:
x_variable = [[LpVariable(f'X(p={p};t={t})', lowBound=0.0, cat='Continuous') for p in p_sets] for t in t_sets]
k_variable = [[LpVariable(f'K(p={p};t={t})', cat='Continuous') for p in p_sets] for t in t_sets]
p_variable = [LpVariable(f'P(t={t})', lowBound=0, cat='Integer') for t in t_sets]
r_variable = [[LpVariable(f'R(p={p};t={t})', cat='Binary') for p in p_sets] for t in t_sets]
e_variable = [[LpVariable(f'E(p={p};t={t})', cat='Binary') for p in p_sets] for t in t_sets]

<h3>Función Objetivo</h3>

<p>La siguiente función objetivo suma el costo total de colocar una orden de abastecimiento, el costo total de mantener el inventario al final de la semana $t$ y, el costo total de penalidades por no cumplir con la meta de inventario</p>

<p>$$\max z = \sum_{p=1}^m\sum_{t=1}^n H_{pt}K_{pt} + \sum_{p=1}^m\sum_{t=1}^n CP_{pt} + \sum_{p=1}^m\sum_{t=1}^n D_{pt}E_{pr}$$</p>


In [5]:
model += lpSum([h_parameter[p] * k_variable[t][p] for t in t_sets for p in p_sets] +
               [c_parameter * p_variable[t] for t in t_sets] + 
               [d_parameter[p] * e_variable[t][p] for t in t_sets for p in p_sets])

<h3>Restricciones:</h3>

<p>El balance de inventario depende de 3 momentos en el horizonte de planeación:</p>
<ol>
    <li>En el periodo inicial, se tiene en cuenta el inventario inicial reportado;</li>
    <li>Durante el leadtime, se tiene en cuenta el inventario al final del periodo anterior como inventario inicial del periodo actual; y</li>
    <li>Despues del leadtime, donde se debe tener en cuenta las ordenes colocadas un leadtime antes</li>
</ol>
<p>Así, las siguientes restricciones modelan el comportamiento descrito:</p>

<p>$K_{pt} = Q_{p} + A_{pt} - F_{pt}$ for $t=1$</p>
<p>$K_{pt} = K_{pt-1} + A_{pt} - F_{pt}$ for $1 < t < L$</p>
<p>$K_{pt} = K_{pt-1} + A_{pt} - F_{pt} + X_{pt-L}$ for $t >= L$</p>

In [6]:
for p in p_sets:
    model += (k_variable[0][p] == q_parameter[p] + a_parameter[0][p] - f_parameter[0][p], 
              f'balanceInvProduct{p}_at_t={0}')

In [7]:
for p in p_sets:
    for t in t_sets[1:l_parameter]:
        model += (k_variable[t][p] == k_variable[t-1][p] + a_parameter[t][p] - f_parameter[t][p],
                  f'balanceInvProduct{p}_at_t={t}')

In [8]:
for p in p_sets:
    for t in t_sets[l_parameter:]:
        model += (k_variable[t][p] == k_variable[t-1][p] + a_parameter[t][p] - f_parameter[t][p] + x_variable[t-l_parameter][p],
                  f'balanceInvProduct{p}_at_t={t}')

<p>La siguiente restricción mantiene el inventario sobre la meta a partir del leadtime</p>
<p>$\sum_{p=L}^{n} X_{pt} >= O_{pt}$ for $t$</p>

In [9]:
for p in p_sets:
    for t in t_sets[l_parameter:]:
        model += (k_variable[t][p] >= o_parameter[t][p] ,f'Inventario del producto {p} sobre la meta en el periodo {t}')

<p>Ahora debemos obligar a que el modelo coloque pedidos sobre el lote mínimo por cada producto, lo que logramos con dos restricciones activadas de manera excluyente con una variable binaria y el parámetro M:</p>
<p>$X_{pt} <= MR_{pt}$</p>
<p>$X_{pt} >= S_{p}R_{pt}$</p>

In [10]:
for p in p_sets:
    for t in t_sets:
        model += (x_variable[t][p] <= m_parameter * r_variable[t][p] ,f'máxima cantidad a pedir del producto {p} en el periodo {t}')
        model += (x_variable[t][p] >= s_parameter[p] * r_variable[t][p] ,f'mínima cantidad a ordenar del producto {p} en el periodo {t} cumpliendo el lóte minimo')

<p>La cantidad a cargar un contenedor, no debe sobrepasar la capacidad del contenedor. Por ahora el parámetro $w_{p}$ nos permite contabilizar qué tanta carga podrá llevar cada contenedor, por lo que el pedido durante el periodo $t$ no debe sobrepasar la capacidad de un contenedor, multiplicado por la cantidad de contenedores</p>
<p>$\sum_{p}^n W_{p}X_{pt} <= T_{t}P_{t}$</p>

In [11]:
for t in t_sets:
    model += (lpSum([w_parameter[p] * x_variable[t][p] for p in p_sets]) <= t_parameter[t] * p_variable[t],
              f'Capacidad del proveedor/contenedores en el periodo{t}')

In [12]:
model.solve()

1

In [13]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 1525504.508104707
E(p=100;t=0): 0.0
E(p=100;t=1): 0.0
E(p=100;t=10): 0.0
E(p=100;t=11): 0.0
E(p=100;t=12): 0.0
E(p=100;t=13): 0.0
E(p=100;t=14): 0.0
E(p=100;t=15): 0.0
E(p=100;t=16): 0.0
E(p=100;t=17): 0.0
E(p=100;t=18): 0.0
E(p=100;t=19): 0.0
E(p=100;t=2): 0.0
E(p=100;t=20): 0.0
E(p=100;t=21): 0.0
E(p=100;t=22): 0.0
E(p=100;t=23): 0.0
E(p=100;t=24): 0.0
E(p=100;t=25): 0.0
E(p=100;t=26): 0.0
E(p=100;t=27): 0.0
E(p=100;t=28): 0.0
E(p=100;t=29): 0.0
E(p=100;t=3): 0.0
E(p=100;t=30): 0.0
E(p=100;t=31): 0.0
E(p=100;t=32): 0.0
E(p=100;t=33): 0.0
E(p=100;t=34): 0.0
E(p=100;t=35): 0.0
E(p=100;t=36): 0.0
E(p=100;t=37): 0.0
E(p=100;t=38): 0.0
E(p=100;t=39): 0.0
E(p=100;t=4): 0.0
E(p=100;t=40): 0.0
E(p=100;t=41): 0.0
E(p=100;t=42): 0.0
E(p=100;t=43): 0.0
E(p=100;t=44): 0.0
E(p=100;t=45): 0.0
E(p=100;t=46): 0.0
E(p=100;t=47): 0.0
E(p=100;t=48): 0.0
E(p=100;t=49): 0.0
E(p=100;t=5): 0.0
E(p=100;t=50): 0.0
E(p=100;t=51): 0.0
E(p=100;t=6): 0.0
E(p=100;t=7): 0.0
E(p=100;t=

E(p=169;t=22): 0.0
E(p=169;t=23): 0.0
E(p=169;t=24): 0.0
E(p=169;t=25): 0.0
E(p=169;t=26): 0.0
E(p=169;t=27): 0.0
E(p=169;t=28): 0.0
E(p=169;t=29): 0.0
E(p=169;t=3): 0.0
E(p=169;t=30): 0.0
E(p=169;t=31): 0.0
E(p=169;t=32): 0.0
E(p=169;t=33): 0.0
E(p=169;t=34): 0.0
E(p=169;t=35): 0.0
E(p=169;t=36): 0.0
E(p=169;t=37): 0.0
E(p=169;t=38): 0.0
E(p=169;t=39): 0.0
E(p=169;t=4): 0.0
E(p=169;t=40): 0.0
E(p=169;t=41): 0.0
E(p=169;t=42): 0.0
E(p=169;t=43): 0.0
E(p=169;t=44): 0.0
E(p=169;t=45): 0.0
E(p=169;t=46): 0.0
E(p=169;t=47): 0.0
E(p=169;t=48): 0.0
E(p=169;t=49): 0.0
E(p=169;t=5): 0.0
E(p=169;t=50): 0.0
E(p=169;t=51): 0.0
E(p=169;t=6): 0.0
E(p=169;t=7): 0.0
E(p=169;t=8): 0.0
E(p=169;t=9): 0.0
E(p=16;t=0): 0.0
E(p=16;t=1): 0.0
E(p=16;t=10): 0.0
E(p=16;t=11): 0.0
E(p=16;t=12): 0.0
E(p=16;t=13): 0.0
E(p=16;t=14): 0.0
E(p=16;t=15): 0.0
E(p=16;t=16): 0.0
E(p=16;t=17): 0.0
E(p=16;t=18): 0.0
E(p=16;t=19): 0.0
E(p=16;t=2): 0.0
E(p=16;t=20): 0.0
E(p=16;t=21): 0.0
E(p=16;t=22): 0.0
E(p=16;t=23): 0.0
E

E(p=246;t=47): 0.0
E(p=246;t=48): 0.0
E(p=246;t=49): 0.0
E(p=246;t=5): 0.0
E(p=246;t=50): 0.0
E(p=246;t=51): 0.0
E(p=246;t=6): 0.0
E(p=246;t=7): 0.0
E(p=246;t=8): 0.0
E(p=246;t=9): 0.0
E(p=247;t=0): 0.0
E(p=247;t=1): 0.0
E(p=247;t=10): 0.0
E(p=247;t=11): 0.0
E(p=247;t=12): 0.0
E(p=247;t=13): 0.0
E(p=247;t=14): 0.0
E(p=247;t=15): 0.0
E(p=247;t=16): 0.0
E(p=247;t=17): 0.0
E(p=247;t=18): 0.0
E(p=247;t=19): 0.0
E(p=247;t=2): 0.0
E(p=247;t=20): 0.0
E(p=247;t=21): 0.0
E(p=247;t=22): 0.0
E(p=247;t=23): 0.0
E(p=247;t=24): 0.0
E(p=247;t=25): 0.0
E(p=247;t=26): 0.0
E(p=247;t=27): 0.0
E(p=247;t=28): 0.0
E(p=247;t=29): 0.0
E(p=247;t=3): 0.0
E(p=247;t=30): 0.0
E(p=247;t=31): 0.0
E(p=247;t=32): 0.0
E(p=247;t=33): 0.0
E(p=247;t=34): 0.0
E(p=247;t=35): 0.0
E(p=247;t=36): 0.0
E(p=247;t=37): 0.0
E(p=247;t=38): 0.0
E(p=247;t=39): 0.0
E(p=247;t=4): 0.0
E(p=247;t=40): 0.0
E(p=247;t=41): 0.0
E(p=247;t=42): 0.0
E(p=247;t=43): 0.0
E(p=247;t=44): 0.0
E(p=247;t=45): 0.0
E(p=247;t=46): 0.0
E(p=247;t=47): 0.0
E(p

E(p=324;t=25): 0.0
E(p=324;t=26): 0.0
E(p=324;t=27): 0.0
E(p=324;t=28): 0.0
E(p=324;t=29): 0.0
E(p=324;t=3): 0.0
E(p=324;t=30): 0.0
E(p=324;t=31): 0.0
E(p=324;t=32): 0.0
E(p=324;t=33): 0.0
E(p=324;t=34): 0.0
E(p=324;t=35): 0.0
E(p=324;t=36): 0.0
E(p=324;t=37): 0.0
E(p=324;t=38): 0.0
E(p=324;t=39): 0.0
E(p=324;t=4): 0.0
E(p=324;t=40): 0.0
E(p=324;t=41): 0.0
E(p=324;t=42): 0.0
E(p=324;t=43): 0.0
E(p=324;t=44): 0.0
E(p=324;t=45): 0.0
E(p=324;t=46): 0.0
E(p=324;t=47): 0.0
E(p=324;t=48): 0.0
E(p=324;t=49): 0.0
E(p=324;t=5): 0.0
E(p=324;t=50): 0.0
E(p=324;t=51): 0.0
E(p=324;t=6): 0.0
E(p=324;t=7): 0.0
E(p=324;t=8): 0.0
E(p=324;t=9): 0.0
E(p=325;t=0): 0.0
E(p=325;t=1): 0.0
E(p=325;t=10): 0.0
E(p=325;t=11): 0.0
E(p=325;t=12): 0.0
E(p=325;t=13): 0.0
E(p=325;t=14): 0.0
E(p=325;t=15): 0.0
E(p=325;t=16): 0.0
E(p=325;t=17): 0.0
E(p=325;t=18): 0.0
E(p=325;t=19): 0.0
E(p=325;t=2): 0.0
E(p=325;t=20): 0.0
E(p=325;t=21): 0.0
E(p=325;t=22): 0.0
E(p=325;t=23): 0.0
E(p=325;t=24): 0.0
E(p=325;t=25): 0.0
E(p

E(p=410;t=31): 0.0
E(p=410;t=32): 0.0
E(p=410;t=33): 0.0
E(p=410;t=34): 0.0
E(p=410;t=35): 0.0
E(p=410;t=36): 0.0
E(p=410;t=37): 0.0
E(p=410;t=38): 0.0
E(p=410;t=39): 0.0
E(p=410;t=4): 0.0
E(p=410;t=40): 0.0
E(p=410;t=41): 0.0
E(p=410;t=42): 0.0
E(p=410;t=43): 0.0
E(p=410;t=44): 0.0
E(p=410;t=45): 0.0
E(p=410;t=46): 0.0
E(p=410;t=47): 0.0
E(p=410;t=48): 0.0
E(p=410;t=49): 0.0
E(p=410;t=5): 0.0
E(p=410;t=50): 0.0
E(p=410;t=51): 0.0
E(p=410;t=6): 0.0
E(p=410;t=7): 0.0
E(p=410;t=8): 0.0
E(p=410;t=9): 0.0
E(p=411;t=0): 0.0
E(p=411;t=1): 0.0
E(p=411;t=10): 0.0
E(p=411;t=11): 0.0
E(p=411;t=12): 0.0
E(p=411;t=13): 0.0
E(p=411;t=14): 0.0
E(p=411;t=15): 0.0
E(p=411;t=16): 0.0
E(p=411;t=17): 0.0
E(p=411;t=18): 0.0
E(p=411;t=19): 0.0
E(p=411;t=2): 0.0
E(p=411;t=20): 0.0
E(p=411;t=21): 0.0
E(p=411;t=22): 0.0
E(p=411;t=23): 0.0
E(p=411;t=24): 0.0
E(p=411;t=25): 0.0
E(p=411;t=26): 0.0
E(p=411;t=27): 0.0
E(p=411;t=28): 0.0
E(p=411;t=29): 0.0
E(p=411;t=3): 0.0
E(p=411;t=30): 0.0
E(p=411;t=31): 0.0
E(p

...

R(p=99;t=19): 0.0
R(p=99;t=2): 0.0
R(p=99;t=20): 0.0
R(p=99;t=21): 0.0
R(p=99;t=22): 0.0
R(p=99;t=23): 0.0
R(p=99;t=24): 0.0
R(p=99;t=25): 0.0
R(p=99;t=26): 0.0
R(p=99;t=27): 0.0
R(p=99;t=28): 0.0
R(p=99;t=29): 0.0
R(p=99;t=3): 0.0
R(p=99;t=30): 0.0
R(p=99;t=31): 0.0
R(p=99;t=32): 0.0
R(p=99;t=33): 0.0
R(p=99;t=34): 0.0
R(p=99;t=35): 0.0
R(p=99;t=36): 0.0
R(p=99;t=37): 0.0
R(p=99;t=38): 0.0
R(p=99;t=39): 0.0
R(p=99;t=4): 0.0
R(p=99;t=40): 0.0
R(p=99;t=41): 0.0
R(p=99;t=42): 0.0
R(p=99;t=43): 0.0
R(p=99;t=44): 0.0
R(p=99;t=45): 0.0
R(p=99;t=46): 0.0
R(p=99;t=47): 0.0
R(p=99;t=48): 0.0
R(p=99;t=49): 0.0
R(p=99;t=5): 0.0
R(p=99;t=50): 0.0
R(p=99;t=51): 0.0
R(p=99;t=6): 0.0
R(p=99;t=7): 0.0
R(p=99;t=8): 0.0
R(p=99;t=9): 0.0
R(p=9;t=0): 1.0
R(p=9;t=1): 0.0
R(p=9;t=10): 0.0
R(p=9;t=11): 0.0
R(p=9;t=12): 0.0
R(p=9;t=13): 0.0
R(p=9;t=14): 0.0
R(p=9;t=15): 0.0
R(p=9;t=16): 0.0
R(p=9;t=17): 0.0
R(p=9;t=18): 0.0
R(p=9;t=19): 0.0
R(p=9;t=2): 0.0
R(p=9;t=20): 0.0
R(p=9;t=21): 0.0
R(p=9;t=22): 0.0
R

X(p=193;t=9): 0.0
X(p=194;t=0): 250.66593
X(p=194;t=1): 0.0
X(p=194;t=10): 0.0
X(p=194;t=11): 0.0
X(p=194;t=12): 0.0
X(p=194;t=13): 0.0
X(p=194;t=14): 0.0
X(p=194;t=15): 0.0
X(p=194;t=16): 0.0
X(p=194;t=17): 0.0
X(p=194;t=18): 0.0
X(p=194;t=19): 0.0
X(p=194;t=2): 0.0
X(p=194;t=20): 0.0
X(p=194;t=21): 0.0
X(p=194;t=22): 0.0
X(p=194;t=23): 0.0
X(p=194;t=24): 0.0
X(p=194;t=25): 0.0
X(p=194;t=26): 0.0
X(p=194;t=27): 0.0
X(p=194;t=28): 0.0
X(p=194;t=29): 0.0
X(p=194;t=3): 0.0
X(p=194;t=30): 0.0
X(p=194;t=31): 0.0
X(p=194;t=32): 0.0
X(p=194;t=33): 0.0
X(p=194;t=34): 0.0
X(p=194;t=35): 0.0
X(p=194;t=36): 0.0
X(p=194;t=37): 0.0
X(p=194;t=38): 0.0
X(p=194;t=39): 0.0
X(p=194;t=4): 0.0
X(p=194;t=40): 0.0
X(p=194;t=41): 0.0
X(p=194;t=42): 0.0
X(p=194;t=43): 0.0
X(p=194;t=44): 0.0
X(p=194;t=45): 0.0
X(p=194;t=46): 0.0
X(p=194;t=47): 0.0
X(p=194;t=48): 0.0
X(p=194;t=49): 0.0
X(p=194;t=5): 0.0
X(p=194;t=50): 0.0
X(p=194;t=51): 0.0
X(p=194;t=6): 0.0
X(p=194;t=7): 0.0
X(p=194;t=8): 0.0
X(p=194;t=9): 0.

X(p=288;t=32): 0.0
X(p=288;t=33): 0.0
X(p=288;t=34): 0.0
X(p=288;t=35): 0.0
X(p=288;t=36): 0.0
X(p=288;t=37): 0.0
X(p=288;t=38): 0.0
X(p=288;t=39): 0.0
X(p=288;t=4): 0.0
X(p=288;t=40): 0.0
X(p=288;t=41): 0.0
X(p=288;t=42): 0.0
X(p=288;t=43): 0.0
X(p=288;t=44): 0.0
X(p=288;t=45): 0.0
X(p=288;t=46): 0.0
X(p=288;t=47): 0.0
X(p=288;t=48): 0.0
X(p=288;t=49): 0.0
X(p=288;t=5): 0.0
X(p=288;t=50): 0.0
X(p=288;t=51): 0.0
X(p=288;t=6): 0.0
X(p=288;t=7): 0.0
X(p=288;t=8): 0.0
X(p=288;t=9): 0.0
X(p=289;t=0): 245.1746
X(p=289;t=1): 0.0
X(p=289;t=10): 0.0
X(p=289;t=11): 0.0
X(p=289;t=12): 0.0
X(p=289;t=13): 0.0
X(p=289;t=14): 0.0
X(p=289;t=15): 0.0
X(p=289;t=16): 0.0
X(p=289;t=17): 0.0
X(p=289;t=18): 0.0
X(p=289;t=19): 0.0
X(p=289;t=2): 0.0
X(p=289;t=20): 0.0
X(p=289;t=21): 0.0
X(p=289;t=22): 0.0
X(p=289;t=23): 0.0
X(p=289;t=24): 0.0
X(p=289;t=25): 0.0
X(p=289;t=26): 0.0
X(p=289;t=27): 0.0
X(p=289;t=28): 0.0
X(p=289;t=29): 0.0
X(p=289;t=3): 0.0
X(p=289;t=30): 0.0
X(p=289;t=31): 0.0
X(p=289;t=32): 0.

X(p=374;t=2): 0.0
X(p=374;t=20): 0.0
X(p=374;t=21): 0.0
X(p=374;t=22): 0.0
X(p=374;t=23): 0.0
X(p=374;t=24): 0.0
X(p=374;t=25): 0.0
X(p=374;t=26): 0.0
X(p=374;t=27): 0.0
X(p=374;t=28): 0.0
X(p=374;t=29): 0.0
X(p=374;t=3): 0.0
X(p=374;t=30): 0.0
X(p=374;t=31): 0.0
X(p=374;t=32): 0.0
X(p=374;t=33): 0.0
X(p=374;t=34): 0.0
X(p=374;t=35): 0.0
X(p=374;t=36): 0.0
X(p=374;t=37): 0.0
X(p=374;t=38): 0.0
X(p=374;t=39): 0.0
X(p=374;t=4): 0.0
X(p=374;t=40): 0.0
X(p=374;t=41): 0.0
X(p=374;t=42): 0.0
X(p=374;t=43): 0.0
X(p=374;t=44): 0.0
X(p=374;t=45): 0.0
X(p=374;t=46): 0.0
X(p=374;t=47): 0.0
X(p=374;t=48): 0.0
X(p=374;t=49): 0.0
X(p=374;t=5): 0.0
X(p=374;t=50): 0.0
X(p=374;t=51): 0.0
X(p=374;t=6): 0.0
X(p=374;t=7): 0.0
X(p=374;t=8): 0.0
X(p=374;t=9): 0.0
X(p=375;t=0): 241.20299
X(p=375;t=1): 0.0
X(p=375;t=10): 0.0
X(p=375;t=11): 0.0
X(p=375;t=12): 0.0
X(p=375;t=13): 0.0
X(p=375;t=14): 0.0
X(p=375;t=15): 0.0
X(p=375;t=16): 0.0
X(p=375;t=17): 0.0
X(p=375;t=18): 0.0
X(p=375;t=19): 0.0
X(p=375;t=2): 0.

X(p=460;t=18): 0.0
X(p=460;t=19): 0.0
X(p=460;t=2): 0.0
X(p=460;t=20): 0.0
X(p=460;t=21): 0.0
X(p=460;t=22): 0.0
X(p=460;t=23): 0.0
X(p=460;t=24): 0.0
X(p=460;t=25): 0.0
X(p=460;t=26): 0.0
X(p=460;t=27): 0.0
X(p=460;t=28): 0.0
X(p=460;t=29): 0.0
X(p=460;t=3): 0.0
X(p=460;t=30): 0.0
X(p=460;t=31): 0.0
X(p=460;t=32): 0.0
X(p=460;t=33): 0.0
X(p=460;t=34): 0.0
X(p=460;t=35): 0.0
X(p=460;t=36): 0.0
X(p=460;t=37): 0.0
X(p=460;t=38): 0.0
X(p=460;t=39): 0.0
X(p=460;t=4): 0.0
X(p=460;t=40): 0.0
X(p=460;t=41): 0.0
X(p=460;t=42): 0.0
X(p=460;t=43): 0.0
X(p=460;t=44): 0.0
X(p=460;t=45): 0.0
X(p=460;t=46): 0.0
X(p=460;t=47): 0.0
X(p=460;t=48): 0.0
X(p=460;t=49): 0.0
X(p=460;t=5): 0.0
X(p=460;t=50): 0.0
X(p=460;t=51): 0.0
X(p=460;t=6): 0.0
X(p=460;t=7): 0.0
X(p=460;t=8): 0.0
X(p=460;t=9): 0.0
X(p=461;t=0): 248.1924
X(p=461;t=1): 0.0
X(p=461;t=10): 0.0
X(p=461;t=11): 0.0
X(p=461;t=12): 0.0
X(p=461;t=13): 0.0
X(p=461;t=14): 0.0
X(p=461;t=15): 0.0
X(p=461;t=16): 0.0
X(p=461;t=17): 0.0
X(p=461;t=18): 0.